# scraping

In [ ]:
import pandas as pd
from googleapiclient.discovery import build


def get_all_replies(comment_id, youtube):
    """Fetch all replies for a given comment ID, handling pagination."""
    replies_data = []
    response = youtube.comments().list(part='snippet', parentId=comment_id, maxResults=100).execute()

    while response:
        for reply in response['items']:
            reply_text = reply['snippet']['textDisplay']

            # Append only the reply text to the list
            replies_data.append([reply_text])

        # Check if there is a next page for replies
        if 'nextPageToken' in response:
            response = youtube.comments().list(part='snippet', parentId=comment_id, pageToken=response['nextPageToken'],
                                               maxResults=100).execute()
        else:
            break

    return replies_data


def video_comments(video_id, api_key):
    # List for storing comments and replies
    comments_data = []

    # Creating youtube resource object
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Retrieve YouTube video results
    video_response = youtube.commentThreads().list(
        part='snippet,replies',
        videoId=video_id,
        maxResults=100
    ).execute()

    # Iterate over video responses to extract comments and replies
    while video_response:

        for item in video_response['items']:
            # Extracting top-level comment information
            top_comment = item['snippet']['topLevelComment']['snippet']
            comment_id = item['snippet']['topLevelComment']['id']
            comment = top_comment['textDisplay']

            # Append only the top-level comment text to the list
            comments_data.append([comment])

            # Check for replies to the top-level comment
            if 'replies' in item:
                # Fetching replies using a separate API call if there are too many
                replies_data = get_all_replies(comment_id, youtube)
                comments_data.extend(replies_data)

        # Pagination: checking if there's a next page of comments
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
                part='snippet,replies',
                videoId=video_id,
                pageToken=video_response['nextPageToken'],
                maxResults=100
            ).execute()
        else:
            break

    return comments_data


# Insert your API key here
api_key = "(type your API))"

# Video ID example: https://www.youtube.com/watch?v=qwWfBoteiy8
video_id = "(type your id example))"

# Call function to get comments and replies
comments = video_comments(video_id, api_key)

# Creating DataFrame from the list of comments and replies (only textDisplay)
df = pd.DataFrame(comments, columns=['textDisplay'])

# Save to CSV file
df.to_csv('1. youtube-comments-with-replies.csv', index=False)
df.head(10)


,textDisplay
0,Aku suka banget sama lagu ini bisa buat hati a...
1,enak di dengar
2,Hidup ku berantakan nadinn😢
3,enek buat tidur langsung nangis
4,P
5,P
6,Kenapa mendadak banget ya aku belum bisa kasih...
7,Ingat mama papa adek😢😢
8,"Orang² kerja buat kebutuhan mereka, sedangkan ..."
9,Laki laki gaboleh cerita ☺️


# Cleaning

In [ ]:
def remove_html_elements(text):
    text = re.sub(r'<br>', ' ', text)  # Mengganti <br> dengan spasi
    text = re.sub(r'<del>', ' ', text)  # Mengganti <del> dengan spasi
    text = re.sub(r'quot', ' ', text)  # Mengganti quot dengan spasi
    return text

# Terapkan fungsi ini ke setiap elemen di DataFrame yang bertipe string
df = pd.read_csv('1. youtube-comments-with-replies.csv')
df['komen_remove_html'] = df['textDisplay'].apply(remove_html_elements)

# Simpan hasil ke file CSV
df.to_csv('2.komen_remove_elemen_html.csv', index=False)

# Tampilkan beberapa baris untuk memastikan
df.loc[100:120]


,textDisplay,komen_remove_html
100,Dah capek,Dah capek
101,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...
102,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...
103,Bu sesungguhnya aku tidak sekuat itu😢😢😢,Bu sesungguhnya aku tidak sekuat itu😢😢😢
104,😇,😇
105,"Pengen pulng ke tuhan, tp juga pengen ketemu a...","Pengen pulng ke tuhan, tp juga pengen ketemu a..."
106,"Aku di bilang beban keluarga, dan kata ibu aku...","Aku di bilang beban keluarga, dan kata ibu aku..."
107,"Kalian jan telat makan yh, kalian jan kek aku,...","Kalian jan telat makan yh, kalian jan kek aku,..."
108,Hidup indah bila mencari berkah<br><br><del>An...,"Hidup indah bila mencari berkah Anjongan, Ka..."
109,Papa ku pergi ke sisi Tuhan gak ada pernah pam...,Papa ku pergi ke sisi Tuhan gak ada pernah pam...


In [ ]:
import pandas as pd  # Import library pandas untuk manipulasi data
import re  # Import library re untuk operasi ekspresi reguler (regex)
import string  # Import library string untuk operasi terkait string

# Fungsi untuk menghilangkan tanda baca, timestamp, username, dan string "<br>" serta "quot" dari teks
def remmovepunc(text):
    if isinstance(text, str):  # Pastikan hanya teks (string) yang diolah
        punc = string.punctuation
        # Hapus tanda baca
        text = text.translate(str.maketrans('', '', punc))
        # Hapus timestamp dalam format menit:detik (misalnya 00:01)
        text = re.sub(r'\b\d{1,2}:\d{2}\b', '', text)
        # Hapus username dalam format @username
        text = re.sub(r'@\w+', '', text)
        text = text.lower()  # Mengubah semua karakter teks menjadi huruf kecil (lower case)
        text = re.sub(r'\d+', '', text)  # Menghapus semua angka dari teks
        text = re.sub('\n', ' ', text)  # Mengganti karakter baris baru (\n) dengan spasi
        text = re.sub(r'\buser\b', ' ', text)  # Menghapus kata "user" yang berdiri sendiri
        text = re.sub(r'[^\w\s]', '', text)  # Menghapus semua tanda baca tetapi mempertahankan huruf dan spasi
        text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(href[^\s]+))', ' ', text)  # Menghapus URL atau link dari teks
        text = re.sub(r'\s+', ' ', text).strip()  # Menghapus spasi berlebih dan memotong spasi di awal/akhir teks
        return text  # Mengembalikan hasil yang sudah diolah

# Terapkan fungsi ini ke setiap elemen di DataFrame yang bertipe string
df = pd.read_csv('2.komen_remove_elemen_html.csv')
df['komen_bersih'] = df['komen_remove_html'].apply(remmovepunc)

# Tampilkan beberapa baris untuk memastikan
df.loc[100:120]


,textDisplay,komen_remove_html,komen_bersih
100,Dah capek,Dah capek,dah capek
101,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...
102,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...
103,Bu sesungguhnya aku tidak sekuat itu😢😢😢,Bu sesungguhnya aku tidak sekuat itu😢😢😢,bu sesungguhnya aku tidak sekuat itu
104,😇,😇,
105,"Pengen pulng ke tuhan, tp juga pengen ketemu a...","Pengen pulng ke tuhan, tp juga pengen ketemu a...",pengen pulng ke tuhan tp juga pengen ketemu an...
106,"Aku di bilang beban keluarga, dan kata ibu aku...","Aku di bilang beban keluarga, dan kata ibu aku...",aku di bilang beban keluarga dan kata ibu aku ...
107,"Kalian jan telat makan yh, kalian jan kek aku,...","Kalian jan telat makan yh, kalian jan kek aku,...",kalian jan telat makan yh kalian jan kek aku k...
108,Hidup indah bila mencari berkah<br><br><del>An...,"Hidup indah bila mencari berkah Anjongan, Ka...",hidup indah bila mencari berkah anjongan kalim...
109,Papa ku pergi ke sisi Tuhan gak ada pernah pam...,Papa ku pergi ke sisi Tuhan gak ada pernah pam...,papa ku pergi ke sisi tuhan gak ada pernah pam...


# Pre-processing

In [ ]:
!pip install pandas nltk

# Tokenize

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import bigrams, trigrams, ngrams

# Download NLTK data (hanya perlu dilakukan sekali)
nltk.download('punkt')

# Fungsi untuk tokenisasi
def tokenisasi(textDisplay):
    if pd.isna(textDisplay):
        return []  # Mengembalikan list kosong jika komentarnya NaN atau None
    return word_tokenize(str(textDisplay))

# Fungsi untuk membuat n-gram
def membuat_ngram(tokens, n):
    return list(ngrams(tokens, n))

# Membaca file CSV
df = pd.read_csv('3.komen_bersih.csv')

# Asumsikan kolom komentar bernama 'komen_bersih'
df['tokenized'] = df['komen_bersih'].apply(tokenisasi)

# Membuat bigram, trigram, dan n-gram (dengan n ditentukan, misalnya n=4)
df['bigrams'] = df['tokenized'].apply(lambda tokenized: membuat_ngram(tokenized, 2))
df['trigrams'] = df['tokenized'].apply(lambda tokenized: membuat_ngram(tokenized, 3))
df['quadgrams'] = df['tokenized'].apply(lambda tokenized: membuat_ngram(tokenized, 4))

# Menampilkan hasil tokenisasi
df[['komen_bersih', 'tokenized']]

df.to_csv('4. komen_token.csv', index=False)

df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,textDisplay,komen_remove_html,komen_bersih,tokenized,bigrams,trigrams,quadgrams
0,Aku suka banget sama lagu ini bisa buat hati a...,Aku suka banget sama lagu ini bisa buat hati a...,aku suka banget sama lagu ini bisa buat hati a...,"[aku, suka, banget, sama, lagu, ini, bisa, bua...","[(aku, suka), (suka, banget), (banget, sama), ...","[(aku, suka, banget), (suka, banget, sama), (b...","[(aku, suka, banget, sama), (suka, banget, sam..."
1,enak di dengar,enak di dengar,enak di dengar,"[enak, di, dengar]","[(enak, di), (di, dengar)]","[(enak, di, dengar)]",[]
2,Hidup ku berantakan nadinn😢,Hidup ku berantakan nadinn😢,hidup ku berantakan nadinn,"[hidup, ku, berantakan, nadinn]","[(hidup, ku), (ku, berantakan), (berantakan, n...","[(hidup, ku, berantakan), (ku, berantakan, nad...","[(hidup, ku, berantakan, nadinn)]"
3,enek buat tidur langsung nangis,enek buat tidur langsung nangis,enek buat tidur langsung nangis,"[enek, buat, tidur, langsung, nangis]","[(enek, buat), (buat, tidur), (tidur, langsung...","[(enek, buat, tidur), (buat, tidur, langsung),...","[(enek, buat, tidur, langsung), (buat, tidur, ..."
4,P,P,p,[p],[],[],[]
...,...,...,...,...,...,...,...
950,Sumpah keren bet lagunya,Sumpah keren bet lagunya,sumpah keren bet lagunya,"[sumpah, keren, bet, lagunya]","[(sumpah, keren), (keren, bet), (bet, lagunya)]","[(sumpah, keren, bet), (keren, bet, lagunya)]","[(sumpah, keren, bet, lagunya)]"
951,"hari ke hari tambah berantakan , sampai lupa a...","hari ke hari tambah berantakan , sampai lupa a...",hari ke hari tambah berantakan sampai lupa apa...,"[hari, ke, hari, tambah, berantakan, sampai, l...","[(hari, ke), (ke, hari), (hari, tambah), (tamb...","[(hari, ke, hari), (ke, hari, tambah), (hari, ...","[(hari, ke, hari, tambah), (ke, hari, tambah, ..."
952,real....,real....,real,[real],[],[],[]
953,Nyentuh,Nyentuh,nyentuh,[nyentuh],[],[],[]


# STEMMING AND LEMMATIZATION

In [ ]:
!pip install PySastrawi

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Pastikan tokenizer dan stopwords data sudah terunduh
nltk.download('punkt')
nltk.download('stopwords')

# Inisialisasi stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stopwords bahasa Indonesia dari NLTK dan tambahan konjungsi
stop_words = set(stopwords.words('indonesian'))
additional_stopwords = {"dan", "atau", "tetapi", "jika", "sementara", "dengan","yang","yg" "pada", "di", "ke", "dari", "untuk", "tentang", "terhadap", "antara", "ke", "lewat", "selama", "di", "dari", "di", "lagi", "lebih", "kemudian", "sekali","tdk","ya","to"}
stop_words.update(additional_stopwords)

# Fungsi untuk membersihkan teks
def clean_text(text):
    # Menghapus karakter non-alfabet
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Mengubah teks menjadi huruf kecil
    text = text.lower()
    # Tokenisasi teks
    words = word_tokenize(text)
    # Menghapus stopwords dan melakukan stemming
    cleaned_words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(cleaned_words)

# Membaca file CSV
df = pd.read_csv('4. komen_token.csv')

# Memastikan kolom 'text' ada di DataFrame, jika kolom bernama lain, sesuaikan dengan nama kolom yang benar
kolom_kalimat = 'tokenized'  # ganti dengan nama kolom yang sesuai

if kolom_kalimat in df.columns:
    # Terapkan fungsi pembersihan ke kolom teks
    df['stemized'] = df[kolom_kalimat].apply(clean_text)

    # Menyimpan DataFrame yang telah dibersihkan ke file CSV baru
    df.to_csv('5. tahap stemming dan lemmatize.csv', index=False)

    # Menampilkan beberapa hasil
    print(df.head())
else:
    print(f"Kolom '{kolom_kalimat}' tidak ditemukan dalam file CSV.")

df.loc[100:120]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                         textDisplay  \
0  Aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                        Hidup ku berantakan nadinn😢   
3                    enek buat tidur langsung nangis   
4                                                  P   

                                   komen_remove_html  \
0  Aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                        Hidup ku berantakan nadinn😢   
3                    enek buat tidur langsung nangis   
4                                                  P   

                                        komen_bersih  \
0  aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                         hidup ku berantakan nadinn   
3                    enek buat tidur langsung nangis   
4                                             

,textDisplay,komen_remove_html,komen_bersih,tokenized,bigrams,trigrams,quadgrams,stemized
100,Dah capek,Dah capek,dah capek,"['dah', 'capek']","[('dah', 'capek')]",[],[],dah capek
101,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...,"['pergi', 'tanpa', 'pamrih', 'pergi', 'tanpa',...","[('pergi', 'tanpa'), ('tanpa', 'pamrih'), ('pa...","[('pergi', 'tanpa', 'pamrih'), ('tanpa', 'pamr...","[('pergi', 'tanpa', 'pamrih', 'pergi'), ('tanp...",pergi pamrih pergi pamit kesanakemari arah set...
102,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...,"['sekedar', 'berandai', 'menatapi', 'diri', 'i...","[('sekedar', 'berandai'), ('berandai', 'menata...","[('sekedar', 'berandai', 'menatapi'), ('berand...","[('sekedar', 'berandai', 'menatapi', 'diri'), ...",dar beranda tatap pencar pergi tuk cari cari
103,Bu sesungguhnya aku tidak sekuat itu😢😢😢,Bu sesungguhnya aku tidak sekuat itu😢😢😢,bu sesungguhnya aku tidak sekuat itu,"['bu', 'sesungguhnya', 'aku', 'tidak', 'sekuat...","[('bu', 'sesungguhnya'), ('sesungguhnya', 'aku...","[('bu', 'sesungguhnya', 'aku'), ('sesungguhnya...","[('bu', 'sesungguhnya', 'aku', 'tidak'), ('ses...",bu sungguh kuat
104,😇,😇,NaN,[],[],[],[],
105,"Pengen pulng ke tuhan, tp juga pengen ketemu a...","Pengen pulng ke tuhan, tp juga pengen ketemu a...",pengen pulng ke tuhan tp juga pengen ketemu an...,"['pengen', 'pulng', 'ke', 'tuhan', 'tp', 'juga...","[('pengen', 'pulng'), ('pulng', 'ke'), ('ke', ...","[('pengen', 'pulng', 'ke'), ('pulng', 'ke', 't...","[('pengen', 'pulng', 'ke', 'tuhan'), ('pulng',...",ken pulng tuhan tp ken ketemu ank dalm kandung
106,"Aku di bilang beban keluarga, dan kata ibu aku...","Aku di bilang beban keluarga, dan kata ibu aku...",aku di bilang beban keluarga dan kata ibu aku ...,"['aku', 'di', 'bilang', 'beban', 'keluarga', '...","[('aku', 'di'), ('di', 'bilang'), ('bilang', '...","[('aku', 'di', 'bilang'), ('di', 'bilang', 'be...","[('aku', 'di', 'bilang', 'beban'), ('di', 'bil...",bilang beban keluarga g dunia bentar ultah tan...
107,"Kalian jan telat makan yh, kalian jan kek aku,...","Kalian jan telat makan yh, kalian jan kek aku,...",kalian jan telat makan yh kalian jan kek aku k...,"['kalian', 'jan', 'telat', 'makan', 'yh', 'kal...","[('kalian', 'jan'), ('jan', 'telat'), ('telat'...","[('kalian', 'jan', 'telat'), ('jan', 'telat', ...","[('kalian', 'jan', 'telat', 'makan'), ('jan', ...",jan telat makan yh jan kek bahagiain orang tua...
108,Hidup indah bila mencari berkah<br><br><del>An...,"Hidup indah bila mencari berkah Anjongan, Ka...",hidup indah bila mencari berkah anjongan kalim...,"['hidup', 'indah', 'bila', 'mencari', 'berkah'...","[('hidup', 'indah'), ('indah', 'bila'), ('bila...","[('hidup', 'indah', 'bila'), ('indah', 'bila',...","[('hidup', 'indah', 'bila', 'mencari'), ('inda...",hidup indah cari berkah anjongan kalimantan ba...
109,Papa ku pergi ke sisi Tuhan gak ada pernah pam...,Papa ku pergi ke sisi Tuhan gak ada pernah pam...,papa ku pergi ke sisi tuhan gak ada pernah pam...,"['papa', 'ku', 'pergi', 'ke', 'sisi', 'tuhan',...","[('papa', 'ku'), ('ku', 'pergi'), ('pergi', 'k...","[('papa', 'ku', 'pergi'), ('ku', 'pergi', 'ke'...","[('papa', 'ku', 'pergi', 'ke'), ('ku', 'pergi'...",papa ku pergi sisi tuhan gak pamit


# Tokenize 2

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Pastikan tokenizer data sudah terunduh
nltk.download('punkt')

# Membaca file CSV
df = pd.read_csv('5. tahap stemming dan lemmatize.csv')

# Memastikan kolom 'kalimat' ada di DataFrame
kolom_kalimat = 'stemized'  # ganti dengan nama kolom yang sesuai

if kolom_kalimat in df.columns:
    # Memastikan semua data dalam kolom adalah string
    df[kolom_kalimat] = df[kolom_kalimat].astype(str)

    # Memisahkan kata dari setiap kalimat dalam kolom yang sesuai
    df['stemized_2'] = df[kolom_kalimat].apply(word_tokenize)

    # Menyimpan DataFrame ke file CSV baru
    df.to_csv('6. tahap_tokenized.csv', index=False)

    # Menampilkan DataFrame dengan kolom kata_kata baru
    print(df.head())
else:
    print(f"Kolom '{kolom_kalimat}' tidak ditemukan dalam file CSV.")

df.loc[100:120]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                         textDisplay  \
0  Aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                        Hidup ku berantakan nadinn😢   
3                    enek buat tidur langsung nangis   
4                                                  P   

                                   komen_remove_html  \
0  Aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                        Hidup ku berantakan nadinn😢   
3                    enek buat tidur langsung nangis   
4                                                  P   

                                        komen_bersih  \
0  aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                         hidup ku berantakan nadinn   
3                    enek buat tidur langsung nangis   
4                                             

,textDisplay,komen_remove_html,komen_bersih,tokenized,bigrams,trigrams,quadgrams,stemized,stemized_2
100,Dah capek,Dah capek,dah capek,"['dah', 'capek']","[('dah', 'capek')]",[],[],dah capek,"[dah, capek]"
101,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...,"['pergi', 'tanpa', 'pamrih', 'pergi', 'tanpa',...","[('pergi', 'tanpa'), ('tanpa', 'pamrih'), ('pa...","[('pergi', 'tanpa', 'pamrih'), ('tanpa', 'pamr...","[('pergi', 'tanpa', 'pamrih', 'pergi'), ('tanp...",pergi pamrih pergi pamit kesanakemari arah set...,"[pergi, pamrih, pergi, pamit, kesanakemari, ar..."
102,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...,"['sekedar', 'berandai', 'menatapi', 'diri', 'i...","[('sekedar', 'berandai'), ('berandai', 'menata...","[('sekedar', 'berandai', 'menatapi'), ('berand...","[('sekedar', 'berandai', 'menatapi', 'diri'), ...",dar beranda tatap pencar pergi tuk cari cari,"[dar, beranda, tatap, pencar, pergi, tuk, cari..."
103,Bu sesungguhnya aku tidak sekuat itu😢😢😢,Bu sesungguhnya aku tidak sekuat itu😢😢😢,bu sesungguhnya aku tidak sekuat itu,"['bu', 'sesungguhnya', 'aku', 'tidak', 'sekuat...","[('bu', 'sesungguhnya'), ('sesungguhnya', 'aku...","[('bu', 'sesungguhnya', 'aku'), ('sesungguhnya...","[('bu', 'sesungguhnya', 'aku', 'tidak'), ('ses...",bu sungguh kuat,"[bu, sungguh, kuat]"
104,😇,😇,NaN,[],[],[],[],nan,[nan]
105,"Pengen pulng ke tuhan, tp juga pengen ketemu a...","Pengen pulng ke tuhan, tp juga pengen ketemu a...",pengen pulng ke tuhan tp juga pengen ketemu an...,"['pengen', 'pulng', 'ke', 'tuhan', 'tp', 'juga...","[('pengen', 'pulng'), ('pulng', 'ke'), ('ke', ...","[('pengen', 'pulng', 'ke'), ('pulng', 'ke', 't...","[('pengen', 'pulng', 'ke', 'tuhan'), ('pulng',...",ken pulng tuhan tp ken ketemu ank dalm kandung,"[ken, pulng, tuhan, tp, ken, ketemu, ank, dalm..."
106,"Aku di bilang beban keluarga, dan kata ibu aku...","Aku di bilang beban keluarga, dan kata ibu aku...",aku di bilang beban keluarga dan kata ibu aku ...,"['aku', 'di', 'bilang', 'beban', 'keluarga', '...","[('aku', 'di'), ('di', 'bilang'), ('bilang', '...","[('aku', 'di', 'bilang'), ('di', 'bilang', 'be...","[('aku', 'di', 'bilang', 'beban'), ('di', 'bil...",bilang beban keluarga g dunia bentar ultah tan...,"[bilang, beban, keluarga, g, dunia, bentar, ul..."
107,"Kalian jan telat makan yh, kalian jan kek aku,...","Kalian jan telat makan yh, kalian jan kek aku,...",kalian jan telat makan yh kalian jan kek aku k...,"['kalian', 'jan', 'telat', 'makan', 'yh', 'kal...","[('kalian', 'jan'), ('jan', 'telat'), ('telat'...","[('kalian', 'jan', 'telat'), ('jan', 'telat', ...","[('kalian', 'jan', 'telat', 'makan'), ('jan', ...",jan telat makan yh jan kek bahagiain orang tua...,"[jan, telat, makan, yh, jan, kek, bahagiain, o..."
108,Hidup indah bila mencari berkah<br><br><del>An...,"Hidup indah bila mencari berkah Anjongan, Ka...",hidup indah bila mencari berkah anjongan kalim...,"['hidup', 'indah', 'bila', 'mencari', 'berkah'...","[('hidup', 'indah'), ('indah', 'bila'), ('bila...","[('hidup', 'indah', 'bila'), ('indah', 'bila',...","[('hidup', 'indah', 'bila', 'mencari'), ('inda...",hidup indah cari berkah anjongan kalimantan ba...,"[hidup, indah, cari, berkah, anjongan, kaliman..."
109,Papa ku pergi ke sisi Tuhan gak ada pernah pam...,Papa ku pergi ke sisi Tuhan gak ada pernah pam...,papa ku pergi ke sisi tuhan gak ada pernah pam...,"['papa', 'ku', 'pergi', 'ke', 'sisi', 'tuhan',...","[('papa', 'ku'), ('ku', 'pergi'), ('pergi', 'k...","[('papa', 'ku', 'pergi'), ('ku', 'pergi', 'ke'...","[('papa', 'ku', 'pergi', 'ke'), ('ku', 'pergi'...",papa ku pergi sisi tuhan gak pamit,"[papa, ku, pergi, sisi, tuhan, gak, pamit]"


# Nomalization Text

In [ ]:
import re
import pandas as pd
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Unduh stopwords bahasa Indonesia dari nltk
nltk.download('stopwords')
nltk_stop_words = set(stopwords.words('indonesian'))

# Stopwords tambahan yang ingin ditambahkan secara manual
# Memuat stopwords tambahan dari file CSV
additional_stop_words = pd.read_csv('stopwords_kata.csv', header=None, sep=",")[0].tolist()
additional_stop_words = set(additional_stop_words)

# Gabungkan kedua set stopwords
stop_words = nltk_stop_words.union(additional_stop_words)

# Baca file CSV dan rename kolomnya
alay_dict = pd.read_csv('kata_slang_normal.csv', header=None, sep=",")
alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})
print(alay_dict.head())

# Pemetaan kamus kata slang dan kata baku
slang_dict = (alay_dict['original'], alay_dict['replacement'])

# Gabungkan kamus kata slang dari file CSV dengan kamus kata slang sebelumnya
# alay_dict_map.update(slang_dict)

# Fungsi untuk normalisasi kata alay
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

# Pemetaan kamus alay
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))

# Fungsi untuk normalisasi kata alay
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

# Fungsi untuk menghapus stopwords
def remove_stopwords(text):
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

# Fungsi untuk stemming
def stemming(text):
    return stemmer.stem(text)

# Fungsi untuk preprocessing teks
def preprocess(text):
    text = normalize_alay(text)
    text = remove_stopwords(text)  # Pindahkan remove_stopwords sebelum stemming
    text = stemming(text)
    return text

# Contoh penggunaan
df = pd.read_csv('6. tahap_tokenized.csv')
df['normalisasi'] = df['stemized_2'].apply(preprocess)

# Menampilkan hasil preprocessing
df[['stemized_2', 'normalisasi']].head()

df.to_csv('7. normalisasi kata slang.csv', index=False)

df


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   original  replacement
0  original  replacement
1         @           di
2      abis        habis
3        ad          ada
4      adlh       adalah


,textDisplay,komen_remove_html,komen_bersih,tokenized,bigrams,trigrams,quadgrams,stemized,stemized_2,normalisasi
0,Aku suka banget sama lagu ini bisa buat hati a...,Aku suka banget sama lagu ini bisa buat hati a...,aku suka banget sama lagu ini bisa buat hati a...,"['aku', 'suka', 'banget', 'sama', 'lagu', 'ini...","[('aku', 'suka'), ('suka', 'banget'), ('banget...","[('aku', 'suka', 'banget'), ('suka', 'banget',...","[('aku', 'suka', 'banget', 'sama'), ('suka', '...",suka banget lagu hati tenang,"['suka', 'banget', 'lagu', 'hati', 'tenang']",suka banget lagu hati tenang
1,enak di dengar,enak di dengar,enak di dengar,"['enak', 'di', 'dengar']","[('enak', 'di'), ('di', 'dengar')]","[('enak', 'di', 'dengar')]",[],enak dengar,"['enak', 'dengar']",enak dengar
2,Hidup ku berantakan nadinn😢,Hidup ku berantakan nadinn😢,hidup ku berantakan nadinn,"['hidup', 'ku', 'berantakan', 'nadinn']","[('hidup', 'ku'), ('ku', 'berantakan'), ('bera...","[('hidup', 'ku', 'berantakan'), ('ku', 'berant...","[('hidup', 'ku', 'berantakan', 'nadinn')]",hidup ku beranta nadinn,"['hidup', 'ku', 'beranta', 'nadinn']",hidup ku beranta nadinn
3,enek buat tidur langsung nangis,enek buat tidur langsung nangis,enek buat tidur langsung nangis,"['enek', 'buat', 'tidur', 'langsung', 'nangis']","[('enek', 'buat'), ('buat', 'tidur'), ('tidur'...","[('enek', 'buat', 'tidur'), ('buat', 'tidur', ...","[('enek', 'buat', 'tidur', 'langsung'), ('buat...",enek tidur langsung nang,"['enek', 'tidur', 'langsung', 'nang']",enek tidur langsung nang
4,P,P,p,['p'],[],[],[],p,['p'],p
...,...,...,...,...,...,...,...,...,...,...
950,Sumpah keren bet lagunya,Sumpah keren bet lagunya,sumpah keren bet lagunya,"['sumpah', 'keren', 'bet', 'lagunya']","[('sumpah', 'keren'), ('keren', 'bet'), ('bet'...","[('sumpah', 'keren', 'bet'), ('keren', 'bet', ...","[('sumpah', 'keren', 'bet', 'lagunya')]",sumpah keren bet lagu,"['sumpah', 'keren', 'bet', 'lagu']",sumpah keren bet lagu
951,"hari ke hari tambah berantakan , sampai lupa a...","hari ke hari tambah berantakan , sampai lupa a...",hari ke hari tambah berantakan sampai lupa apa...,"['hari', 'ke', 'hari', 'tambah', 'berantakan',...","[('hari', 'ke'), ('ke', 'hari'), ('hari', 'tam...","[('hari', 'ke', 'hari'), ('ke', 'hari', 'tamba...","[('hari', 'ke', 'hari', 'tambah'), ('ke', 'har...",beranta lupa tuju dewasa beban pikir mimpi ter...,"['beranta', 'lupa', 'tuju', 'dewasa', 'beban',...",beranta lupa tuju dewasa beban pikir mimpi ter...
952,real....,real....,real,['real'],[],[],[],real,['real'],real
953,Nyentuh,Nyentuh,nyentuh,['nyentuh'],[],[],[],nyentuh,['nyentuh'],nyentuh


# PREPROCESSING

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Unduh stopwords bahasa Indonesia dari nltk
nltk.download('stopwords')
nltk_stop_words = set(stopwords.words('indonesian'))

# Stopwords tambahan yang ingin ditambahkan secara manual
manual_stopwords = {"banget","ku", "a","ga", "g", "yh", "o", "sih", "biar", "pas", "dan", "atau", "tetapi", "jika", "sementara", "dengan",
 "yang", "yg", "pada", "di", "ke", "dari", "untuk", "nya","nan","udh","NaN"
 "tentang", "terhadap", "antara", "lewat", "selama", "lagi", "lebih", "kemudian", "sekali", "tdk", "ya", "to"}

# Gabungkan stopwords NLTK dengan stopwords tambahan manual
stop_words = nltk_stop_words.union(manual_stopwords)

# Memuat stopwords tambahan dari file CSV
additional_stop_words = pd.read_csv('stopwords_kata.csv', header=None, sep=",")[0].tolist()
additional_stop_words = set(additional_stop_words)

# Gabungkan semua set stopwords
stop_words = stop_words.union(additional_stop_words)

# Baca file CSV untuk kamus alay dan rename kolomnya
alay_dict = pd.read_csv('kata_slang_normal.csv', header=None, sep=",")
alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})

# Pemetaan kamus kata slang dan kata baku
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))

# Fungsi untuk normalisasi kata alay
def normalize_alay(text):
    words = text.split()
    normalized_words = [alay_dict_map[word] if word in alay_dict_map else word for word in words]
    return ' '.join(normalized_words)

# Fungsi untuk menghapus stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Fungsi untuk stemming
def stemming(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Fungsi untuk preprocessing teks
def preprocess(text):
    text = normalize_alay(text)
    text = remove_stopwords(text)
    text = stemming(text)
    return text

kolom_kalimat = 'normalisasi'  # ganti dengan nama kolom yang sesuai

if kolom_kalimat in df.columns:
    df[kolom_kalimat] = df[kolom_kalimat].astype(str)
    # Terapkan fungsi pembersihan ke kolom teks
    df['lemma'] = df[kolom_kalimat].apply(preprocess)

    # Menyimpan DataFrame yang telah dibersihkan ke file CSV baru
    df.to_csv('8. preprocessing komen.csv', index=False)

    # Menampilkan beberapa hasil
    print(df.head())
else:
    print(f"Kolom '{kolom_kalimat}' tidak ditemukan dalam file CSV.")
df.loc[100:120]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                         textDisplay  \
0  Aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                        Hidup ku berantakan nadinn😢   
3                    enek buat tidur langsung nangis   
4                                                  P   

                                   komen_remove_html  \
0  Aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                        Hidup ku berantakan nadinn😢   
3                    enek buat tidur langsung nangis   
4                                                  P   

                                        komen_bersih  \
0  aku suka banget sama lagu ini bisa buat hati a...   
1                                     enak di dengar   
2                         hidup ku berantakan nadinn   
3                    enek buat tidur langsung nangis   
4                                             

,textDisplay,komen_remove_html,komen_bersih,tokenized,bigrams,trigrams,quadgrams,stemized,stemized_2,normalisasi,lemma
100,Dah capek,Dah capek,dah capek,"['dah', 'capek']","[('dah', 'capek')]",[],[],dah capek,"['dah', 'capek']",dah capek,deh capek
101,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...,pergi tanpa pamrih pergi tanpa pamit akan kesa...,"['pergi', 'tanpa', 'pamrih', 'pergi', 'tanpa',...","[('pergi', 'tanpa'), ('tanpa', 'pamrih'), ('pa...","[('pergi', 'tanpa', 'pamrih'), ('tanpa', 'pamr...","[('pergi', 'tanpa', 'pamrih', 'pergi'), ('tanp...",pergi pamrih pergi pamit kesanakemari arah set...,"['pergi', 'pamrih', 'pergi', 'pamit', 'kesanak...",pergi pamrih pergi pamit kesanakemari arah set...,pergi pamrih pergi pamit kesanakemari arah set...
102,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...,sekedar berandai menatapi diri ini berpencar p...,"['sekedar', 'berandai', 'menatapi', 'diri', 'i...","[('sekedar', 'berandai'), ('berandai', 'menata...","[('sekedar', 'berandai', 'menatapi'), ('berand...","[('sekedar', 'berandai', 'menatapi', 'diri'), ...",dar beranda tatap pencar pergi tuk cari cari,"['dar', 'beranda', 'tatap', 'pencar', 'pergi',...",dar beranda tatap pencar pergi tuk cari cari,dar beranda tatap pencar pergi tuk cari cari
103,Bu sesungguhnya aku tidak sekuat itu😢😢😢,Bu sesungguhnya aku tidak sekuat itu😢😢😢,bu sesungguhnya aku tidak sekuat itu,"['bu', 'sesungguhnya', 'aku', 'tidak', 'sekuat...","[('bu', 'sesungguhnya'), ('sesungguhnya', 'aku...","[('bu', 'sesungguhnya', 'aku'), ('sesungguhnya...","[('bu', 'sesungguhnya', 'aku', 'tidak'), ('ses...",bu sungguh kuat,"['bu', 'sungguh', 'kuat']",bu sungguh kuat,bu sungguh kuat
104,😇,😇,NaN,[],[],[],[],NaN,['nan'],nan,
105,"Pengen pulng ke tuhan, tp juga pengen ketemu a...","Pengen pulng ke tuhan, tp juga pengen ketemu a...",pengen pulng ke tuhan tp juga pengen ketemu an...,"['pengen', 'pulng', 'ke', 'tuhan', 'tp', 'juga...","[('pengen', 'pulng'), ('pulng', 'ke'), ('ke', ...","[('pengen', 'pulng', 'ke'), ('pulng', 'ke', 't...","[('pengen', 'pulng', 'ke', 'tuhan'), ('pulng',...",ken pulng tuhan tp ken ketemu ank dalm kandung,"['ken', 'pulng', 'tuhan', 'tp', 'ken', 'ketemu...",ken pulng tuhan tp ken ketemu ank dalm kandung,ken pulng tuhan ken ketemu anak dalm kandung
106,"Aku di bilang beban keluarga, dan kata ibu aku...","Aku di bilang beban keluarga, dan kata ibu aku...",aku di bilang beban keluarga dan kata ibu aku ...,"['aku', 'di', 'bilang', 'beban', 'keluarga', '...","[('aku', 'di'), ('di', 'bilang'), ('bilang', '...","[('aku', 'di', 'bilang'), ('di', 'bilang', 'be...","[('aku', 'di', 'bilang', 'beban'), ('di', 'bil...",bilang beban keluarga g dunia bentar ultah tan...,"['bilang', 'beban', 'keluarga', 'g', 'dunia', ...",bilang beban keluarga g dunia bentar ultah tan...,bilang beban keluarga dunia sebentar ultah tan...
107,"Kalian jan telat makan yh, kalian jan kek aku,...","Kalian jan telat makan yh, kalian jan kek aku,...",kalian jan telat makan yh kalian jan kek aku k...,"['kalian', 'jan', 'telat', 'makan', 'yh', 'kal...","[('kalian', 'jan'), ('jan', 'telat'), ('telat'...","[('kalian', 'jan', 'telat'), ('jan', 'telat', ...","[('kalian', 'jan', 'telat', 'makan'), ('jan', ...",jan telat makan yh jan kek bahagiain orang tua...,"['jan', 'telat', 'makan', 'yh', 'jan', 'kek', ...",jan telat makan yh jan kek bahagiain orang tua...,jan lambat makan jan bahagiain orang tua jan l...
108,Hidup indah bila mencari berkah<br><br><del>An...,"Hidup indah bila mencari berkah Anjongan, Ka...",hidup indah bila mencari berkah anjongan kalim...,"['hidup', 'indah', 'bila', 'mencari', 'berkah'...","[('hidup', 'indah'), ('indah', 'bila'), ('bila...","[('hidup', 'indah', 'bila'), ('indah', 'bila',...","[('hidup', 'indah', 'bila', 'mencari'), ('inda...",hidup indah cari berkah anjongan kalimantan ba...,"['hidup', 'indah', 'cari', 'berkah', 'anjongan...",hidup indah cari berkah anjongan kal

# Removal Rare Word

In [ ]:
import pandas as pd
from collections import Counter

# Load the CSV file
data = pd.read_csv('8. preprocessing komen.csv')

# Function to remove rare words
def remove_rare_words(text, rare_words):
    return ' '.join([word for word in text.split() if word not in rare_words])

# Filter rows where 'lemma' contains more than 5 words
filtered_data = data[data['lemma'].apply(lambda x: isinstance(x, str) and len(x.split()) > 10)]

# Get the word frequencies across all 'lemma' texts
all_words = ' '.join(filtered_data['lemma']).split()
word_freq = Counter(all_words)

# Define rare words as those appearing less than a threshold (e.g., 5 times)
rare_words = {word for word, count in word_freq.items() if count <10}

# Remove rare words from the 'lemma' column
filtered_data['lemma'] = filtered_data['lemma'].apply(lambda x: remove_rare_words(x, rare_words))

# Save the cleaned data to a new CSV file
filtered_data.to_csv('9. remove_rare_word.csv', index=False)

# Display the filtered dataframe
filtered_data.loc[100:120]


<ipython-input-240-5fe3ba10e50e>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['lemma'] = filtered_data['lemma'].apply(lambda x: remove_rare_words(x, rare_words))


,textDisplay,komen_remove_html,komen_bersih,tokenized,bigrams,trigrams,quadgrams,stemized,stemized_2,normalisasi,lemma
106,"Aku di bilang beban keluarga, dan kata ibu aku...","Aku di bilang beban keluarga, dan kata ibu aku...",aku di bilang beban keluarga dan kata ibu aku ...,"['aku', 'di', 'bilang', 'beban', 'keluarga', '...","[('aku', 'di'), ('di', 'bilang'), ('bilang', '...","[('aku', 'di', 'bilang'), ('di', 'bilang', 'be...","[('aku', 'di', 'bilang', 'beban'), ('di', 'bil...",bilang beban keluarga g dunia bentar ultah tan...,"['bilang', 'beban', 'keluarga', 'g', 'dunia', ...",bilang beban keluarga g dunia bentar ultah tan...,keluarga dunia tinggal capek sedih mu
107,"Kalian jan telat makan yh, kalian jan kek aku,...","Kalian jan telat makan yh, kalian jan kek aku,...",kalian jan telat makan yh kalian jan kek aku k...,"['kalian', 'jan', 'telat', 'makan', 'yh', 'kal...","[('kalian', 'jan'), ('jan', 'telat'), ('telat'...","[('kalian', 'jan', 'telat'), ('jan', 'telat', ...","[('kalian', 'jan', 'telat', 'makan'), ('jan', ...",jan telat makan yh jan kek bahagiain orang tua...,"['jan', 'telat', 'makan', 'yh', 'jan', 'kek', ...",jan telat makan yh jan kek bahagiain orang tua...,orang tua
120,"pas kecil lihat televisi ,lihat siaran tentang...","pas kecil lihat televisi ,lihat siaran tentang...",pas kecil lihat televisi lihat siaran tentang ...,"['pas', 'kecil', 'lihat', 'televisi', 'lihat',...","[('pas', 'kecil'), ('kecil', 'lihat'), ('lihat...","[('pas', 'kecil', 'lihat'), ('kecil', 'lihat',...","[('pas', 'kecil', 'lihat', 'televisi'), ('keci...",pas lihat televisi lihat siar tiri jahatterus ...,"['pas', 'lihat', 'televisi', 'lihat', 'siar', ...",pas lihat televisi lihat siar tiri jahatterus ...,lihat lihat lihat


In [ ]:
import pandas as pd
from collections import Counter

# Load the CSV file
data = pd.read_csv('9. remove_rare_word.csv')

# Filter rows where 'lemma' contains more than 5 words
filtered_data = data[data['lemma'].apply(lambda x: isinstance(x, str) and len(x.split()) > 5)]

# Get all the words in the 'lemma' column
all_words = ' '.join(filtered_data['lemma']).split()

# Count the frequency of each word
word_freq = Counter(all_words)

# Convert the word frequencies to a DataFrame for better visualization
freq_df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])

# Sort the DataFrame by frequency in descending order
freq_df = freq_df.sort_values(by='Frequency', ascending=False)

# Display the top 20 most frequent words
top_20_frequent_words = freq_df.head(20)
freq_df.head(20)

,Word,Frequency
39,orang,40
18,hidup,32
9,anak,29
28,kakak,25
33,mama,22
30,sakit,18
19,dewasa,18
42,pulang,18
26,capek,18
10,dunia,18


# Drop Kolom

In [ ]:
filtered_data.columns
komen_fix= pd.read_csv('9. remove_rare_word.csv')
komen_fix.drop(columns=['komen_remove_html',	'komen_bersih',
                        'tokenized',	'bigrams',	'trigrams',	'quadgrams',	'stemized',
                        'stemized_2',	'normalisasi',	], inplace=True)
komen_fix.rename(columns={'textDisplay': 'sebelum', 'lemma': 'sesudah'}, inplace=True)
komen_fix.to_csv('10. Preprocessing final', index=False)
komen_fix.head()


,sebelum,sesudah
0,Kenapa mendadak banget ya aku belum bisa kasih...,moga yaa salah gak suka hati bahagia yaa salah
1,&quot;jangan gaduh adik beradik&quot; <br>&quo...,adik adik senang anak dunia anak bahagia tenan...
2,lihat komenannya berasa bukan aku sendiri<br>t...,lihat teman rumah
3,dri semuanya komen yg ku baca adalah hal yg pl...,sedih coba semangat yaa yaa semangat
4,ayah aku 3 hari yang lalu meninggal ibukku 7 t...,ayah tinggal tinggal tinggal ayah cerita cerit...


In [ ]:
#TAGGING
# Download the Indonesian model for Stanza
!pip install stanza



In [ ]:
import stanza
# Initialize the pipeline
nlp = stanza.Pipeline('id', processors='tokenize,pos')


def tag_text(text):

  if isinstance(text, str):
    doc = nlp(text)
    tagged_text = ' '.join([f'{word.text}/{word.upos}' for sent in doc.sentences for word in sent.words])
    return tagged_text
  else:
    return text

# Apply the tagging function to the 'textDisplay' column
komen_fix['tagged_komen'] = komen_fix['lemma'].apply(tag_text)
komen_fix.head()

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: id (Indonesian):
| Processor | Package    |
--------------------------
| tokenize  | gsd        |
| mwt       | gsd        |
| pos       | gsd_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


,textDisplay,lemma,tagged_komen
0,Kenapa mendadak banget ya aku belum bisa kasih...,moga yaa salah gak suka hati bahagia yaa salah,moga/X yaa/PART salah/ADJ gak/PART suka/VERB h...
1,&quot;jangan gaduh adik beradik&quot; <br>&quo...,adik adik senang anak dunia anak bahagia tenan...,adik/NOUN adik/NOUN senang/ADJ anak/NOUN dunia...
2,lihat komenannya berasa bukan aku sendiri<br>t...,lihat teman rumah,lihat/VERB teman/NOUN rumah/NOUN
3,dri semuanya komen yg ku baca adalah hal yg pl...,sedih coba semangat yaa yaa semangat,sedih/ADJ coba/VERB semangat/NOUN yaa/PART yaa...
4,ayah aku 3 hari yang lalu meninggal ibukku 7 t...,ayah tinggal tinggal tinggal ayah cerita cerit...,ayah/NOUN tinggal/ADV tinggal/ADV tinggal/VERB...


In [ ]:
#CHUNKING
#import stanza

# Download the Indonesian model for Stanza if not already downloaded
#stanza.download('id')

# Initialize the pipeline
#nlp = stanza.Pipeline('id', processors='tokenize,pos')


def chunk_text(text):

    if isinstance(text, str):
        doc = nlp(text)
        noun_chunks = []
        current_chunk = []
        for sentence in doc.sentences:
            for word in sentence.words:
                # Identify nouns, adjectives, proper nouns, and include possessive pronouns
                if word.upos in ['NOUN', 'ADJ', 'PROPN', 'PRON', 'VERB'] or word.deprel == 'case':
                    current_chunk.append(word.text)
                else:
                    if current_chunk:
                        noun_chunks.append(" ".join(current_chunk))
                        current_chunk = []
            if current_chunk:
                noun_chunks.append(" ".join(current_chunk))
        return " | ".join(noun_chunks)
    else:
        return text


# Apply the chunking function to the 'textDisplay' column and create a new column 'textDisplay_chunked'
komen_fix['komen_chunked'] = komen_fix['lemma'].apply(chunk_text)

komen_fix.head(50)


,textDisplay,lemma,tagged_komen,komen_chunked
0,Kenapa mendadak banget ya aku belum bisa kasih...,moga yaa salah gak suka hati bahagia yaa salah,moga/X yaa/PART salah/ADJ gak/PART suka/VERB h...,salah | suka hati bahagia | salah
1,&quot;jangan gaduh adik beradik&quot; <br>&quo...,adik adik senang anak dunia anak bahagia tenan...,adik/NOUN adik/NOUN senang/ADJ anak/NOUN dunia...,adik adik senang anak dunia anak bahagia tenan...
2,lihat komenannya berasa bukan aku sendiri<br>t...,lihat teman rumah,lihat/VERB teman/NOUN rumah/NOUN,lihat teman rumah
3,dri semuanya komen yg ku baca adalah hal yg pl...,sedih coba semangat yaa yaa semangat,sedih/ADJ coba/VERB semangat/NOUN yaa/PART yaa...,sedih coba semangat | semangat
4,ayah aku 3 hari yang lalu meninggal ibukku 7 t...,ayah tinggal tinggal tinggal ayah cerita cerit...,ayah/NOUN tinggal/ADV tinggal/ADV tinggal/VERB...,ayah | tinggal ayah cerita cerita bahagia ting...
5,Hahaa jaman dulu itu yaa aku kira dewasa begit...,yaa dewasa senang anak dewasa dewasa dunia,yaa/PART dewasa/ADJ senang/ADJ anak/NOUN dewas...,dewasa senang anak dewasa dewasa dunia
6,part tersedih ketika teman mu selalu kamu apre...,sedih teman mu tuhan teman,sedih/ADJ teman/NOUN mu/PRON tuhan/NOUN teman/...,sedih teman mu tuhan teman
7,"Andai gue masih memeluk lu selain merangkul, b...",luka luka rumah rumah rumah teman bahagia kelu...,luka/NOUN luka/NOUN rumah/NOUN rumah/NOUN ruma...,luka luka rumah rumah rumah teman bahagia kelu...
8,Pengen nya hidup seperti sekarang tapi keluarg...,ken hidup keluarga hidup gak hidup hidup,ken/NOUN hidup/VERB keluarga/NOUN hidup/ADJ ga...,ken hidup keluarga hidup | hidup hidup
9,"hey, aku umur 12th aku mau cerita cerita aja d...",cerita cerita anak capek anak anak capek nilai...,cerita/NOUN cerita/NOUN anak/NOUN capek/NOUN a...,cerita cerita anak capek anak anak capek nilai...
